# Case study for CLEAN vs DMLF
## 1. Import packages

In [1]:
import os, sys
import pandas as pd
sys.path.append("./")
sys.path.append("./src/CLEAN/")
# from utils import *
from utils import prepare_infer_fasta, table2fasta
from infer import infer_maxsep
from sklearn.metrics import precision_score,accuracy_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
%load_ext autoreload
%autoreload 2

# Common function
#format EC resluts from clean
def format_res(res_file):
    def format_ec(eclist):
        res = [item.split('/')[0].replace('EC:','')  for item in eclist if item!=None]
        res = ';'.join(res)
        return res
    
    with open(res_file, 'r') as f:
        data = [line.strip().split(',') for line in f.readlines()]
    data = pd.DataFrame(data)
    data['ec_clean'] = data.apply(lambda x : format_ec(eclist=x[1:].to_list()), axis=1)
    data=data[[0,'ec_clean']].rename(columns={0:'Entry'})
    return data

def eva_score(groundtruth, pred, dataname, format_length=20):
    pre = precision_score(groundtruth, pred, average='weighted', zero_division=0)
    rec = recall_score(groundtruth, pred, average='weighted')
    f1 = f1_score(groundtruth, pred, average='weighted')
    acc = accuracy_score(groundtruth, pred)
    print(f'{dataname: <{format_length}}\t{round(acc,6): <{format_length}}\t{round(pre,6): <{format_length}}\t{round(rec,6): <{format_length}}\t{round(f1,6)}')

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecstr=ecstr.replace(' ','')
    ecs = ecstr.split(';')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

In [2]:
# data_36case = pd.read_excel('/home/shizhenkun/codebase/CLEAN/app/data/36case.xlsx')
# table2fasta(table=data_36case[['Uniprot ID', 'seq']], file_out='/home/shizhenkun/codebase/CLEAN/app/data/36case.fasta')
# data_36case.head(2)

## 2. Prepare infer data

In [3]:
# # price datasets 149 sequences
# prepare_infer_fasta('price')

# # new 392 dataset
# prepare_infer_fasta('new')

# # 2022年4月以后的数据
# prepare_infer_fasta('new2257up')

# mannualy 36case
# prepare_infer_fasta('36case')

## 3. Inferance

In [4]:
# infer_maxsep('split100', 'new', report_metrics=True, pretrained=True)

# infer_maxsep('split100', 'price', report_metrics=True, pretrained=True)

# infer_maxsep('split100', 'new2257up', report_metrics=True, pretrained=True)

# infer_maxsep('split100', '36case', report_metrics=True, pretrained=True)

In [5]:
# #price
# ! /home/shizhenkun/miniconda3/envs/DMLF/bin/python /home/shizhenkun/codebase/DMLF/production.py -i /home/shizhenkun/codebase/CLEAN/app/data/price.fasta -o /home/shizhenkun/codebase/CLEAN/app/results/price_dmlf.csv -mode p

# #new
# ! /home/shizhenkun/miniconda3/envs/DMLF/bin/python /home/shizhenkun/codebase/DMLF/production.py -i /home/shizhenkun/codebase/CLEAN/app/data/new.fasta -o /home/shizhenkun/codebase/CLEAN/app/results/new_dmlf.csv -mode p

# #new2257up
# ! /home/shizhenkun/miniconda3/envs/DMLF/bin/python /home/shizhenkun/codebase/DMLF/production.py -i /home/shizhenkun/codebase/CLEAN/app/data/new2257up.fasta -o /home/shizhenkun/codebase/CLEAN/app/results/new2257up_dmlf.csv -mode p

# #36case
# ! /home/shizhenkun/miniconda3/envs/DMLF/bin/python /home/shizhenkun/codebase/DMLF/production.py -i /home/shizhenkun/codebase/CLEAN/app/data/36case.fasta -o /home/shizhenkun/codebase/CLEAN/app/results/36case_dmlf.csv -mode p

## 4. Analysis

In [12]:
res_price_dmlf = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/results/price_dmlf.csv', sep='\t').rename(columns={'id_input':'Entry', 'ec_pred':'ec_dmlf'})
res_new_dmlf = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/results/new_dmlf.csv', sep='\t').rename(columns={'id_input':'Entry', 'ec_pred':'ec_dmlf'})
res_new2257up_dmlf = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/results/new2257up_dmlf.csv', sep='\t').rename(columns={'id_input':'Entry', 'ec_pred':'ec_dmlf'})
res_36case_dmlf = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/results/36case_dmlf.csv', sep='\t').rename(columns={'id_input':'Entry', 'ec_pred':'ec_dmlf'})

In [13]:
# original data
data_price = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/data/datasets/price.csv', sep='\t').rename(columns={'EC number':'ec_groundtruth'})
data_new = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/data/datasets/new.csv', sep='\t').rename(columns={'EC number':'ec_groundtruth'})
data_new2257up = pd.read_csv('/home/shizhenkun/codebase/CLEAN/app/data/datasets/new2257up.tsv', sep='\t').rename(columns={'EC number':'ec_groundtruth'})
data_new2257up = data_new2257up.fillna('-')
data_new2257up['ec_groundtruth'] = data_new2257up['ec_groundtruth'].apply(lambda x: specific_ecs(ecstr=x))

data_36case = pd.read_excel('/home/shizhenkun/codebase/CLEAN/app/data/36case.xlsx').rename(columns={'Uniprot ID':'Entry'})

#predicted results
res_price = format_res(res_file='/home/shizhenkun/codebase/CLEAN/app/results/price_maxsep.csv')
res_new = format_res(res_file='/home/shizhenkun/codebase/CLEAN/app/results/new_maxsep.csv')
res_new2257up = format_res(res_file='/home/shizhenkun/codebase/CLEAN/app/results/new2257up_maxsep.csv')
res_36case = format_res(res_file='/home/shizhenkun/codebase/CLEAN/app/results/36case_maxsep.csv')

# merege resluts
res_price = data_price.merge(res_price, on='Entry', how='left').merge(res_price_dmlf, on='Entry', how='left')
res_new = data_new.merge(res_new, on='Entry', how='left').merge(res_new_dmlf, on='Entry', how='left')
res_new2257up = data_new2257up.merge(res_new2257up, on='Entry', how='left').merge(res_new2257up_dmlf, on='Entry', how='left')
res_36case = data_36case.merge(res_36case, on='Entry', how='left').merge(res_36case_dmlf, on='Entry', how='left')


#judgement
res_price['right_clean'] = res_price.apply(lambda x : 1 if x.ec_groundtruth==x.ec_clean else 0 ,axis=1)
res_new['right_clean'] = res_new.apply(lambda x : 1 if x.ec_groundtruth==x.ec_clean else 0 ,axis=1)
res_new2257up['right_clean'] = res_new2257up.apply(lambda x : 1 if x.ec_groundtruth==x.ec_clean else 0 ,axis=1)
res_36case['right_clean'] = res_36case.apply(lambda x : 1 if x.ec_groundtruth==x.ec_clean else 0 ,axis=1)

res_price['right_dmlf'] = res_price.apply(lambda x : 1 if x.ec_groundtruth==x.ec_dmlf else 0 ,axis=1)
res_new['right_dmlf'] = res_new.apply(lambda x : 1 if x.ec_groundtruth==x.ec_dmlf else 0 ,axis=1)
res_new2257up['right_dmlf'] = res_new2257up.apply(lambda x : 1 if x.ec_groundtruth==x.ec_dmlf else 0 ,axis=1)
res_36case['right_dmlf'] = res_36case.apply(lambda x : 1 if x.ec_groundtruth==x.ec_dmlf else 0 ,axis=1)

res_36case.head(3)

,index,Type of halogenase,Name,Entry,CLEAN,status,seq,DeepEC,BLASTpa,ProteInferb,...,ECPredc,COFACTORc,ec_groundtruth,"Manual curationd,h",文献,备注,ec_clean,ec_dmlf,right_clean,right_dmlf
0,1,Haloperoxidase,NapH1,A7KH27,1.11.1.10,NaN,MTTSGHTSFSDYSPRRRSMLLGGLGGAAALSAAGFTGMASASPRGS...,n.p.e,6.1.1.19,1.-.-.-,...,3.-.-.-,1.11.1.10,1.11.1.10,1.11.1.10 (57),NaN,NaN,1.11.1.10,1.11.1.10,1,1
1,2,Haloperoxidase,MarH1,A0A0F7N9T7,1.11.1.10,NaN,MTTGHSPVSGFSPRRRSLLIGGASAAALLPLGHAGTAAAAEGGKAA...,n.p.,unsuref,1.-.-.-,...,3.-.-.-,1.11.1.18,1.11.1.10,1.11.1.10 (58),NaN,NaN,1.11.1.10,1.11.1.10,1,1
2,3,Haloperoxidase,MarH2,A0A559V0A1,1.11.1.10,deleted,MTTSGSSSVPGFSPRRRSLLLGGGSAAALTALGHAGTAAAEPGPAA...,n.p.,n.p.,3.1.-.-,...,1.13.-.-,1.11.1.18,1.11.1.10,1.11.1.10 (58),Meroterpenoid natural products from Streptomyc...,NaN,1.11.1.10,3.4.21.105,1,0


## 5. Output Metrics Score

In [14]:
format_length=20

print(f'{"":<{format_length}}\t{"Accuracy":<{format_length}}\t{"Precision":<{format_length}}\t{"Recall":<{format_length}}\t{"F1":<{format_length}}')

eva_score(res_price.ec_groundtruth, res_price.ec_clean, 'price_149_clean')
eva_score(res_price.ec_groundtruth, res_price.ec_dmlf, 'price_149_dmlf')
print('\n')
eva_score(res_new.ec_groundtruth, res_new.ec_clean, 'new_392_clean')
eva_score(res_new.ec_groundtruth, res_new.ec_dmlf, 'new_392_dmlf')
print('\n')

eva_score(res_new2257up.ec_groundtruth.fillna('-'), res_new2257up.ec_clean, 'new2257up_clean')
eva_score(res_new2257up.ec_groundtruth.fillna('-'), res_new2257up.ec_dmlf, 'new2257up_dmlf')
print('\n')
eva_score(res_36case.ec_groundtruth.fillna('-'), res_36case.ec_clean, '36case_clean')
eva_score(res_36case.ec_groundtruth.fillna('-'), res_36case.ec_dmlf, 'case_dmlf')

                    	Accuracy            	Precision           	Recall              	F1                  
price_149_clean     	0.33557             	0.46868             	0.33557             	0.366623
price_149_dmlf      	0.14094             	0.175296            	0.14094             	0.147519


new_392_clean       	0.441327            	0.539456            	0.441327            	0.465093
new_392_dmlf        	0.811224            	0.831633            	0.811224            	0.816893


new2257up_clean     	0.096588            	0.080542            	0.096588            	0.081001
new2257up_dmlf      	0.920248            	0.917514            	0.920248            	0.917875


36case_clean        	0.277778            	0.234954            	0.277778            	0.245062
case_dmlf           	0.222222            	0.388889            	0.222222            	0.282407


In [17]:
with pd.ExcelWriter('/home/shizhenkun/codebase/CLEAN/app/results/all_cases.xlsx') as writer:
    res_price.to_excel(writer, encoding='utf-8', sheet_name='price_data_149')
    res_new.to_excel(writer, encoding='utf-8', sheet_name='new_data_392' )
    res_new2257up.to_excel(writer,encoding='utf-8', sheet_name='data_after_202204')
    res_36case.to_excel(writer, encoding='utf-8',sheet_name='36_manual_data')